# ArangoDB + LangChain

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/arangodb/interactive_tutorials/blob/master/notebooks/Langchain.ipynb)

Large language models (LLMs) are emerging as a transformative technology, enabling developers to build applications that they previously could not. However, using these LLMs in isolation is often insufficient for creating a truly powerful app - the real power comes when you can combine them with other sources of computation or knowledge.

[LangChain](https://www.langchain.com/) is a framework for developing applications powered by language models. It enables applications that are:
- Data-aware: connect a language model to other sources of data
- Agentic: allow a language model to interact with its environment

On July 25 2023, ArangoDB introduced the first release of the [ArangoGraphQAChain](https://langchain-langchain.vercel.app/docs/integrations/providers/arangodb) to the LangChain community, allowing you to leverage LLMs to provide a natural language interface for your ArangoDB data.

**Please note**: This notebook uses the LangChain `ChatOpenAI` wrapper, which requires you to have a **paid** [OpenAI API Key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key). However, other Chat Models are available as well: https://github.com/langchain-ai/langchain/tree/master/libs/langchain/langchain/chat_models

## Setup

In [1]:
%%capture

# 1: Install the dependencies

!pip install python-arango # The ArangoDB Python Driver
!pip install adb-cloud-connector # The ArangoDB Cloud Instance provisioner
!pip install openai==0.28.1
!pip install langchain==0.0.314

In [4]:
# 2: Provision a temporary ArangoDB Cloud instance

from adb_cloud_connector import get_temp_credentials

connection = get_temp_credentials(tutorialName="LangChain")

connection

Log: requesting new credentials...
Succcess: new credentials acquired


{'dbName': 'TUT5hi70sjppn72exqhqumy8a',
 'username': 'TUT9vlxw42ppe5kp9i4kaenfa',
 'password': 'TUTs0ft2thu7tkyi88fi0zs',
 'hostname': 'tutorials.arangodb.cloud',
 'port': 8529,
 'url': 'https://tutorials.arangodb.cloud:8529'}

In [5]:
# 3: Instantiate the ArangoDB-Python driver

from arango import ArangoClient

client = ArangoClient(hosts=connection["url"])

db = client.db(
    connection["dbName"],
    connection["username"],
    connection["password"],
    verify=True
)

db

<StandardDatabase TUT5hi70sjppn72exqhqumy8a>

In [6]:
# 4: Load sample data
# We'll be relying on our Game Of Thrones dataset, representing the parent-child
# relationships of certain characters from the GoT universe

if db.has_graph("GameOfThrones"):
    db.delete_graph("GameOfThrones", drop_collections=True)


edge_definitions=[
    {
        "edge_collection": "ChildOf",
        "from_vertex_collections": ["Characters"],
        "to_vertex_collections": ["Characters"],
    }
]

documents = [
    # Starks (8)
    {"_key": "RickardStark", "name": "Rickard", "surname": "Stark", "alive": False, "age": 60, "gender": "male"},
    {"_key": "LyarraStark", "name": "Lyarra", "surname": "Stark", "alive": False, "age": 60, "gender": "female"},
    {"_key": "NedStark", "name": "Ned", "surname": "Stark", "alive": True, "age": 41, "gender": "male"},
    {"_key": "CatelynStark", "name": "Catelyn", "surname": "Stark", "alive": False, "age": 40, "gender": "female"},
    {"_key": "AryaStark", "name": "Arya", "surname": "Stark", "alive": True, "age": 11, "gender": "female"},
    {"_key": "BranStark", "name": "Bran", "surname": "Stark", "alive": True, "age": 10, "gender": "male"},
    { "_key": "RobbStark", "name": "Robb", "surname": "Stark", "alive": False, "age": 16, "gender": "male"},
    { "_key": "SansaStark", "name": "Sansa", "surname": "Stark", "alive": True, "age": 13, "gender": "female"},

    # Lannisters (4)
    { "_key": "TywinLannister", "name": "Tywin", "surname": "Lannister", "alive": False, "age": 67, "gender": "male" },
    { "_key": "JaimeLannister", "name": "Jaime", "surname": "Lannister", "alive": True, "age": 36, "gender": "male" },
    { "_key": "CerseiLannister", "name": "Cersei", "surname": "Lannister", "alive": True, "age": 36, "gender": "female" },
    { "_key": "TyrionLannister", "name": "Tyrion", "surname": "Lannister", "alive": True, "age": 32, "gender": "male" },

    # Baratheons (1)
    { "_key": "JoffreyBaratheon", "name": "Joffrey", "surname": "Baratheon", "alive": False, "age": 19, "gender": "male"},
]

edges = [
    {"_to": "Characters/NedStark", "_from": "Characters/BranStark"},
    {"_to": "Characters/NedStark", "_from": "Characters/RobbStark" },
    {"_to": "Characters/NedStark", "_from": "Characters/SansaStark" },
    {"_to": "Characters/NedStark", "_from": "Characters/AryaStark" },
    {"_to": "Characters/CatelynStark", "_from": "Characters/AryaStark"},
    {"_to": "Characters/CatelynStark", "_from": "Characters/BranStark"},
    {"_to": "Characters/CatelynStark", "_from": "Characters/RobbStark" },
    {"_to": "Characters/CatelynStark", "_from": "Characters/SansaStark" },
    {"_to": "Characters/RickardStark", "_from": "Characters/NedStark"},
    {"_to": "Characters/LyarraStark", "_from": "Characters/NedStark"},

    {"_to": "Characters/TywinLannister", "_from": "Characters/JaimeLannister" },
    {"_to": "Characters/TywinLannister", "_from": "Characters/CerseiLannister" },
    {"_to": "Characters/TywinLannister", "_from": "Characters/TyrionLannister" },
    {"_to": "Characters/CerseiLannister", "_from": "Characters/JoffreyBaratheon" },
    {"_to": "Characters/JaimeLannister", "_from": "Characters/JoffreyBaratheon" }
]

db.delete_graph("GameOfThrones", ignore_missing=True, drop_collections=True)
db.create_graph("GameOfThrones", edge_definitions)
db.collection("Characters").import_bulk(documents)
db.collection("ChildOf").import_bulk(edges)

{'error': False,
 'created': 15,
 'errors': 0,
 'empty': 0,
 'updated': 0,
 'ignored': 0,
 'details': []}

In [12]:
# 5: Instantiate the ArangoDB-LangChain Graph wrapper

from langchain.graphs import ArangoGraph

graph = ArangoGraph(db)

graph.schema

{'Graph Schema': [{'graph_name': 'GameOfThrones',
   'edge_definitions': [{'edge_collection': 'ChildOf',
     'from_vertex_collections': ['Characters'],
     'to_vertex_collections': ['Characters']}]}],
 'Collection Schema': [{'collection_name': 'Characters',
   'collection_type': 'document',
   'document_properties': [{'name': '_key', 'type': 'str'},
    {'name': '_id', 'type': 'str'},
    {'name': '_rev', 'type': 'str'},
    {'name': 'name', 'type': 'str'},
    {'name': 'surname', 'type': 'str'},
    {'name': 'alive', 'type': 'bool'},
    {'name': 'age', 'type': 'int'},
    {'name': 'gender', 'type': 'str'}],
   'example_document': {'_key': 'RickardStark',
    '_id': 'Characters/RickardStark',
    '_rev': '_gw7CBWu---',
    'name': 'Rickard',
    'surname': 'Stark',
    'alive': False,
    'age': 60,
    'gender': 'male'}},
  {'collection_name': 'ChildOf',
   'collection_type': 'edge',
   'edge_properties': [{'name': '_key', 'type': 'str'},
    {'name': '_id', 'type': 'str'},
    {'n

In [13]:
# 6: Set your OpenAI API Key
# https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [14]:
# 7: Instantiate the OpenAI Chat model
# Note that other models can be used as well
# Ref: https://github.com/langchain-ai/langchain/tree/master/libs/langchain/langchain/chat_models

from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name='gpt-4')

In [15]:
# 8: Instantiate the LangChain Question-Answering Chain with
# our **model** and **graph**

from langchain.chains import ArangoGraphQAChain

chain = ArangoGraphQAChain.from_llm(model, graph=graph, verbose=True)

## Prompting

In [16]:
chain.run("Who are the 2 youngest characters?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters
FOR character IN Characters
SORT character.age ASC
LIMIT 2
RETURN character

AQL Result:
[{'_key': 'BranStark', '_id': 'Characters/BranStark', '_rev': '_gw7CBWu--D', 'name': 'Bran', 'surname': 'Stark', 'alive': True, 'age': 10, 'gender': 'male'}, {'_key': 'AryaStark', '_id': 'Characters/AryaStark', '_rev': '_gw7CBWu--C', 'name': 'Arya', 'surname': 'Stark', 'alive': True, 'age': 11, 'gender': 'female'}]

> Finished chain.


'The two youngest characters in the Game of Thrones database are Bran Stark, a 10-year-old male, and Arya Stark, an 11-year-old female. Both characters are currently alive.'

In [17]:
chain.run("How are Bran Stark and Arya Stark related?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v, e, p IN 1..10 ANY 'Characters/BranStark' ChildOf
  FILTER p.vertices[-1]._key == 'AryaStark'
  RETURN p

AQL Result:
[{'vertices': [{'_key': 'BranStark', '_id': 'Characters/BranStark', '_rev': '_gw7CBWu--D', 'name': 'Bran', 'surname': 'Stark', 'alive': True, 'age': 10, 'gender': 'male'}, {'_key': 'CatelynStark', '_id': 'Characters/CatelynStark', '_rev': '_gw7CBWu--B', 'name': 'Catelyn', 'surname': 'Stark', 'alive': False, 'age': 40, 'gender': 'female'}, {'_key': 'SansaStark', '_id': 'Characters/SansaStark', '_rev': '_gw7CBWu--F', 'name': 'Sansa', 'surname': 'Stark', 'alive': True, 'age': 13, 'gender': 'female'}, {'_key': 'NedStark', '_id': 'Characters/NedStark', '_rev': '_gw7CBWu--A', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'gender': 'male'}, {'_key': 'AryaStark', '_id': 'Characters/AryaStark', '_rev': '_gw7CBWu--C', 'name': 'Arya', 'surname': 'Stark', 'alive': True, 'age': 

'Bran Stark and Arya Stark are siblings. They are both children of Ned Stark and Catelyn Stark.'

In [18]:
chain.run("Who are Bran Stark’s grandparents?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v, e IN 2..2 OUTBOUND 'Characters/BranStark' ChildOf
RETURN v.name

AQL Result:
['Lyarra', 'Rickard']

> Finished chain.


"Bran Stark's grandparents are Lyarra and Rickard."

In [19]:
chain.run("Fetch me the character count for each family")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters
FOR doc IN Characters
COLLECT family = doc.surname WITH COUNT INTO characterCount
RETURN { "family": family, "characterCount": characterCount }

AQL Result:
[{'family': 'Baratheon', 'characterCount': 1}, {'family': 'Lannister', 'characterCount': 4}, {'family': 'Stark', 'characterCount': 8}]

> Finished chain.


'The Game of Thrones database shows that there is 1 character from the Baratheon family, 4 characters from the Lannister family, and 8 characters from the Stark family.'

In [20]:
chain.run("What is the age difference between Rickard Stark and Arya Stark?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters
LET RickardAge = (
  FOR character IN Characters
    FILTER character._key == 'RickardStark'
    RETURN character.age
)[0]
LET AryaAge = (
  FOR character IN Characters
    FILTER character._key == 'AryaStark'
    RETURN character.age
)[0]
RETURN RickardAge - AryaAge

AQL Result:
[49]

> Finished chain.


'The age difference between Rickard Stark and Arya Stark is 49 years.'

In [21]:
chain.run("Wie alt ist Rickard Stark?") # (German: "How old is Rickard Stark?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters
FOR character IN Characters
    FILTER character.name == 'Rickard' AND character.surname == 'Stark'
    RETURN character.age

AQL Result:
[60]

> Finished chain.


'Rickard Stark is 60 years old.'

In [22]:
chain.run("What is the average age within the Stark family?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters
FOR character IN Characters
    FILTER character.surname == 'Stark'
    COLLECT AGGREGATE avgAge = AVG(character.age)
    RETURN avgAge

AQL Result:
[31.375]

> Finished chain.


'The average age of the Stark family members in the Game of Thrones database is approximately 31.4 years.'

In [23]:
chain.run("Does Bran Stark have a dead parent?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v, e IN 1..1 OUTBOUND 'Characters/BranStark' ChildOf
FILTER v.alive == false
RETURN v

AQL Result:
[{'_key': 'CatelynStark', '_id': 'Characters/CatelynStark', '_rev': '_gw7CBWu--B', 'name': 'Catelyn', 'surname': 'Stark', 'alive': False, 'age': 40, 'gender': 'female'}]

> Finished chain.


'Yes, Bran Stark does have a deceased parent. His mother, Catelyn Stark, is no longer alive.'

In [26]:
chain.run("Who are Catelyn Stark's children?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v IN Characters
    FILTER v.name == 'Catelyn' AND v.surname == 'Stark'
    FOR e IN ChildOf
        FILTER e._to == v._id
        FOR c IN Characters
            FILTER c._id == e._from
            RETURN c

AQL Result:
[{'_key': 'AryaStark', '_id': 'Characters/AryaStark', '_rev': '_gw7CBWu--C', 'name': 'Arya', 'surname': 'Stark', 'alive': True, 'age': 11, 'gender': 'female'}, {'_key': 'BranStark', '_id': 'Characters/BranStark', '_rev': '_gw7CBWu--D', 'name': 'Bran', 'surname': 'Stark', 'alive': True, 'age': 10, 'gender': 'male'}, {'_key': 'RobbStark', '_id': 'Characters/RobbStark', '_rev': '_gw7CBWu--E', 'name': 'Robb', 'surname': 'Stark', 'alive': False, 'age': 16, 'gender': 'male'}, {'_key': 'SansaStark', '_id': 'Characters/SansaStark', '_rev': '_gw7CBWu--F', 'name': 'Sansa', 'surname': 'Stark', 'alive': True, 'age': 13, 'gender': 'female'}]

> Finished chain.


'Catelyn Stark has four children. Arya Stark, a female, is 11 years old and currently alive. Bran Stark, a male, is 10 years old and also alive. Robb Stark, a male, was 16 years old but is now deceased. Lastly, Sansa Stark, a female, is 13 years old and currently alive.'

In [27]:
chain.run("Add Jon Snow, 31, a male character")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters
INSERT {
  "_key": "JonSnow",
  "name": "Jon",
  "surname": "Snow",
  "alive": true,
  "age": 31,
  "gender": "male"
} INTO Characters

AQL Result:
[]

> Finished chain.


'Summary:\nThe character Jon Snow, a 31-year-old male, has been successfully added to the database.'

In [29]:
chain.run("Create a ChildOf edge from Jon Snow to Ned Stark")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
INSERT {_from: "Characters/JonSnow", _to: "Characters/NedStark"} INTO ChildOf

AQL Result:
[]

> Finished chain.


'Summary:\n\nA relationship has been successfully established in the Game of Thrones graph, indicating that Jon Snow is a child of Ned Stark.'

In [30]:
chain.run("Who is related to Ned Stark?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v, e IN 1..1 ANY 'Characters/NedStark' ChildOf
RETURN v

AQL Result:
[{'_key': 'JonSnow', '_id': 'Characters/JonSnow', '_rev': '_gw7E4km--A', 'name': 'Jon', 'surname': 'Snow', 'alive': True, 'age': 31, 'gender': 'male'}, {'_key': 'AryaStark', '_id': 'Characters/AryaStark', '_rev': '_gw7CBWu--C', 'name': 'Arya', 'surname': 'Stark', 'alive': True, 'age': 11, 'gender': 'female'}, {'_key': 'SansaStark', '_id': 'Characters/SansaStark', '_rev': '_gw7CBWu--F', 'name': 'Sansa', 'surname': 'Stark', 'alive': True, 'age': 13, 'gender': 'female'}, {'_key': 'RobbStark', '_id': 'Characters/RobbStark', '_rev': '_gw7CBWu--E', 'name': 'Robb', 'surname': 'Stark', 'alive': False, 'age': 16, 'gender': 'male'}, {'_key': 'BranStark', '_id': 'Characters/BranStark', '_rev': '_gw7CBWu--D', 'name': 'Bran', 'surname': 'Stark', 'alive': True, 'age': 10, 'gender': 'male'}, {'_key': 'JonSnow', '_id': 'Characters/JonSnow', '_rev

'Ned Stark is related to several characters. His children are Jon Snow, a 31-year-old male who is alive; Arya Stark, an 11-year-old female who is alive; Sansa Stark, a 13-year-old female who is alive; Robb Stark, a 16-year-old male who is deceased; and Bran Stark, a 10-year-old male who is alive. He is also related to Lyarra Stark, a 60-year-old female who is deceased, and Rickard Stark, a 60-year-old male who is deceased.'

In [31]:
chain.run("What can you tell me about the characters?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters
FOR character IN Characters
    RETURN character

AQL Result:
[{'_key': 'RickardStark', '_id': 'Characters/RickardStark', '_rev': '_gw7CBWu---', 'name': 'Rickard', 'surname': 'Stark', 'alive': False, 'age': 60, 'gender': 'male'}, {'_key': 'LyarraStark', '_id': 'Characters/LyarraStark', '_rev': '_gw7CBWu--_', 'name': 'Lyarra', 'surname': 'Stark', 'alive': False, 'age': 60, 'gender': 'female'}, {'_key': 'NedStark', '_id': 'Characters/NedStark', '_rev': '_gw7CBWu--A', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'gender': 'male'}, {'_key': 'CatelynStark', '_id': 'Characters/CatelynStark', '_rev': '_gw7CBWu--B', 'name': 'Catelyn', 'surname': 'Stark', 'alive': False, 'age': 40, 'gender': 'female'}, {'_key': 'AryaStark', '_id': 'Characters/AryaStark', '_rev': '_gw7CBWu--C', 'name': 'Arya', 'surname': 'Stark', 'alive': True, 'age': 11, 'gender': 'female'}, {'_key': 'BranStark', '_id': 'Characters/BranS

'The database contains information about several characters. Here are some of them:\n\n1. Rickard Stark, a male character who is not alive. He was 60 years old.\n2. Lyarra Stark, a female character who is also not alive. She was 60 years old.\n3. Ned Stark, a male character who is alive and 41 years old.\n4. Catelyn Stark, a female character who is not alive. She was 40 years old.\n5. Arya Stark, a female character who is alive and 11 years old.\n6. Bran Stark, a male character who is alive and 10 years old.\n7. Robb Stark, a male character who is not alive. He was 16 years old.\n8. Sansa Stark, a female character who is alive and 13 years old.\n9. Tywin Lannister, a male character who is not alive. He was 67 years old.\n10. Jaime Lannister, a male character who is alive and 36 years old.'

In [32]:
chain.run("What is the shortest path from Bran Stark to Rickard Stark?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v, e IN ANY SHORTEST_PATH 'Characters/BranStark' TO 'Characters/RickardStark' ChildOf
RETURN {vertices: v, edges: e}

AQL Result:
[{'vertices': {'_key': 'BranStark', '_id': 'Characters/BranStark', '_rev': '_gw7CBWu--D', 'name': 'Bran', 'surname': 'Stark', 'alive': True, 'age': 10, 'gender': 'male'}, 'edges': None}, {'vertices': {'_key': 'NedStark', '_id': 'Characters/NedStark', '_rev': '_gw7CBWu--A', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'gender': 'male'}, 'edges': {'_key': '266199645213', '_id': 'ChildOf/266199645213', '_from': 'Characters/BranStark', '_to': 'Characters/NedStark', '_rev': '_gw7CBZW---'}}, {'vertices': {'_key': 'RickardStark', '_id': 'Characters/RickardStark', '_rev': '_gw7CBWu---', 'name': 'Rickard', 'surname': 'Stark', 'alive': False, 'age': 60, 'gender': 'male'}, 'edges': {'_key': '266199645221', '_id': 'ChildOf/266199645221', '_from': 'Characters/NedStark

'The shortest path from Bran Stark to Rickard Stark in the Game of Thrones universe is through Ned Stark. Bran Stark is the son of Ned Stark, who in turn is the son of Rickard Stark.'

In [39]:
chain.run("What is th family tree of Joffrey Baratheon?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v, e, p IN 1..100 OUTBOUND 'Characters/JoffreyBaratheon' ChildOf
RETURN p.vertices

AQL Result:
[[{'_key': 'JoffreyBaratheon', '_id': 'Characters/JoffreyBaratheon', '_rev': '_gw7CBWu--L', 'name': 'Joffrey', 'surname': 'Baratheon', 'alive': False, 'age': 19, 'gender': 'male'}, {'_key': 'JaimeLannister', '_id': 'Characters/JaimeLannister', '_rev': '_gw7CBWu--H', 'name': 'Jaime', 'surname': 'Lannister', 'alive': True, 'age': 36, 'gender': 'male'}], [{'_key': 'JoffreyBaratheon', '_id': 'Characters/JoffreyBaratheon', '_rev': '_gw7CBWu--L', 'name': 'Joffrey', 'surname': 'Baratheon', 'alive': False, 'age': 19, 'gender': 'male'}, {'_key': 'JaimeLannister', '_id': 'Characters/JaimeLannister', '_rev': '_gw7CBWu--H', 'name': 'Jaime', 'surname': 'Lannister', 'alive': True, 'age': 36, 'gender': 'male'}, {'_key': 'TywinLannister', '_id': 'Characters/TywinLannister', '_rev': '_gw7CBWu--G', 'name': 'Tywin', 'surna

'The family tree of Joffrey Baratheon includes his parents and his grandfather. His father is Jaime Lannister, who is currently alive and 36 years old. His mother is Cersei Lannister, who is also alive and 36 years old. His grandfather is Tywin Lannister, who is no longer alive and was 67 years old.'

In [40]:
chain.run("What is the relationship between Bran Stark and Rickard Stark?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v, e IN 1..10 ANY 'Characters/BranStark' ChildOf
  FILTER v._key == 'RickardStark'
  RETURN e

AQL Result:
[{'_key': '266199645221', '_id': 'ChildOf/266199645221', '_from': 'Characters/NedStark', '_to': 'Characters/RickardStark', '_rev': '_gw7CBZe--C'}, {'_key': '266199645221', '_id': 'ChildOf/266199645221', '_from': 'Characters/NedStark', '_to': 'Characters/RickardStark', '_rev': '_gw7CBZe--C'}, {'_key': '266199645221', '_id': 'ChildOf/266199645221', '_from': 'Characters/NedStark', '_to': 'Characters/RickardStark', '_rev': '_gw7CBZe--C'}, {'_key': '266199645221', '_id': 'ChildOf/266199645221', '_from': 'Characters/NedStark', '_to': 'Characters/RickardStark', '_rev': '_gw7CBZe--C'}, {'_key': '266199645221', '_id': 'ChildOf/266199645221', '_from': 'Characters/NedStark', '_to': 'Characters/RickardStark', '_rev': '_gw7CBZe--C'}, {'_key': '266199645221', '_id': 'ChildOf/266199645221', '_from': 'Charact

'Bran Stark and Rickard Stark are related through Ned Stark. Ned Stark is the son of Rickard Stark and the father of Bran Stark.'

In [43]:
chain.run("Are Arya Stark and Ned Stark related?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v, e IN 1..10 ANY 'Characters/AryaStark' ChildOf
  FILTER v._key == 'NedStark'
  RETURN v

AQL Result:
[{'_key': 'NedStark', '_id': 'Characters/NedStark', '_rev': '_gw7CBWu--A', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'gender': 'male'}, {'_key': 'NedStark', '_id': 'Characters/NedStark', '_rev': '_gw7CBWu--A', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'gender': 'male'}, {'_key': 'NedStark', '_id': 'Characters/NedStark', '_rev': '_gw7CBWu--A', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'gender': 'male'}, {'_key': 'NedStark', '_id': 'Characters/NedStark', '_rev': '_gw7CBWu--A', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'gender': 'male'}, {'_key': 'NedStark', '_id': 'Characters/NedStark', '_rev': '_gw7CBWu--A', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'gender': 'male'}, {'_key': 'NedStark', '_id': 'Characte

'Yes, Arya Stark and Ned Stark are related. Ned Stark, who is alive and 41 years old, is a relative of Arya Stark.'

In [44]:
chain.run("Is Ned Stark alive?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters
FOR character IN Characters
    FILTER character._key == 'NedStark'
    RETURN character.alive

AQL Result:
[True]

> Finished chain.


'Yes, according to the database, Ned Stark is alive.'

In [46]:
chain.run("Ned Stark has died. Update the data")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters
UPDATE "NedStark" WITH { alive: false } IN Characters

AQL Result:
[]

> Finished chain.


'The data has been updated successfully. Ned Stark is now marked as deceased in the database.'

In [48]:
chain.run("How many characters are alive? How many characters are dead?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters
FOR character IN Characters
    COLLECT status = character.alive WITH COUNT INTO count
    RETURN { "status": status, "count": count }

AQL Result:
[{'status': False, 'count': 7}, {'status': True, 'count': 7}]

> Finished chain.


'In the Game of Thrones database, there are 7 characters who are currently alive and 7 characters who are deceased.'

In [52]:
chain.run("Is Arya Stark an orphan?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v, e IN 1..1 ANY 'Characters/AryaStark' ChildOf
  FILTER v.alive == false
  RETURN v

AQL Result:
[{'_key': 'CatelynStark', '_id': 'Characters/CatelynStark', '_rev': '_gw7CBWu--B', 'name': 'Catelyn', 'surname': 'Stark', 'alive': False, 'age': 40, 'gender': 'female'}, {'_key': 'NedStark', '_id': 'Characters/NedStark', '_rev': '_gw7KMqO--_', 'name': 'Ned', 'surname': 'Stark', 'alive': False, 'age': 41, 'gender': 'male'}]

> Finished chain.


'Yes, Arya Stark is an orphan. Both of her parents, Catelyn Stark and Ned Stark, are deceased.'

## Prompt Modifiers

You can alter the values of the following `ArangoDBGraphQAChain` class variables to modify the behaviour of your chain results


In [54]:
# Notice how the following prompt returns nothing;
chain.run("Who are the grandchildren of Rickard Stark?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v, e IN 2..2 OUTBOUND 'Characters/RickardStark' ChildOf
RETURN v

AQL Result:
[]

> Finished chain.


'Based on the information available, Rickard Stark does not have any grandchildren recorded in the database.'

In [55]:
# A simple reminder to use INBOUND (instead of OUTBOUND) returns the correct result;
chain.run("Who are the grandchildren of Rickard Stark? Remember to use INBOUND")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v, e IN 2..2 INBOUND 'Characters/RickardStark' GRAPH 'GameOfThrones'
RETURN v

AQL Result:
[{'_key': 'JonSnow', '_id': 'Characters/JonSnow', '_rev': '_gw7E4km--A', 'name': 'Jon', 'surname': 'Snow', 'alive': True, 'age': 31, 'gender': 'male'}, {'_key': 'AryaStark', '_id': 'Characters/AryaStark', '_rev': '_gw7CBWu--C', 'name': 'Arya', 'surname': 'Stark', 'alive': True, 'age': 11, 'gender': 'female'}, {'_key': 'SansaStark', '_id': 'Characters/SansaStark', '_rev': '_gw7CBWu--F', 'name': 'Sansa', 'surname': 'Stark', 'alive': True, 'age': 13, 'gender': 'female'}, {'_key': 'RobbStark', '_id': 'Characters/RobbStark', '_rev': '_gw7CBWu--E', 'name': 'Robb', 'surname': 'Stark', 'alive': False, 'age': 16, 'gender': 'male'}, {'_key': 'BranStark', '_id': 'Characters/BranStark', '_rev': '_gw7CBWu--D', 'name': 'Bran', 'surname': 'Stark', 'alive': True, 'age': 10, 'gender': 'male'}]

> Finished chain.


'The grandchildren of Rickard Stark are Jon Snow, Arya Stark, Sansa Stark, Robb Stark, and Bran Stark. Jon Snow is 31 years old and is alive. Arya Stark is 11 years old and is also alive. Sansa Stark is 13 years old and is alive. Robb Stark was 16 years old but is no longer alive. Bran Stark is 10 years old and is alive.'

In [57]:
# We can solidify this pattern by making using of **chain.aql_examples**

# The AQL Examples modifier instructs the LLM to adapt its AQL-completion style
# to the user’s examples. These examples arepassed to the AQL Generation Prompt
# Template to promote few-shot-learning.

chain.aql_examples = """
# Who are the grandchildren of Rickard Stark?
WITH Characters, ChildOf
FOR v, e IN 2..2 INBOUND 'Characters/RickardStark' ChildOf
  RETURN v

# Is Ned Stark alive?
RETURN DOCUMENT('Characters/NedStark').alive
"""

# Note how we are no longer specifying the use of INBOUND
chain.run("Who is the grandchildren of Tywin Lannister?")



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH Characters, ChildOf
FOR v, e IN 2..2 INBOUND 'Characters/TywinLannister' ChildOf
  RETURN v

AQL Result:
[{'_key': 'JoffreyBaratheon', '_id': 'Characters/JoffreyBaratheon', '_rev': '_gw7CBWu--L', 'name': 'Joffrey', 'surname': 'Baratheon', 'alive': False, 'age': 19, 'gender': 'male'}, {'_key': 'JoffreyBaratheon', '_id': 'Characters/JoffreyBaratheon', '_rev': '_gw7CBWu--L', 'name': 'Joffrey', 'surname': 'Baratheon', 'alive': False, 'age': 19, 'gender': 'male'}]

> Finished chain.


'The grandchildren of Tywin Lannister is Joffrey Baratheon.'

In [58]:
# Other modifiers include:

# Specify the maximum number of AQL Query Results to return
chain.top_k = 5

# Specify the maximum amount of AQL Generation attempts that should be made
# before returning an error
chain.max_aql_generation_attempts = 5

# Specify whether or not to return the AQL Query in the output dictionary
# Use `chain("...")` instead of `chain.run("...")` to see this change
chain.return_aql_query = True

# Specify whether or not to return the AQL JSON Result in the output dictionary
# Use `chain("...")` instead of `chain.run("...")` to see this change
chain.return_aql_result = True

## Prompting via Gradio

In [59]:
%%capture
!pip install gradio

In [60]:
import gradio as gr

demo = gr.Interface(chain.run, inputs=gr.Textbox(label="Chat"), outputs=gr.Textbox(label="Output"))

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5c781b651452226c9f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
